In [15]:
!pip install gradio faker openai pandas matplotlib seaborn

In [21]:
import gradio as gr
import pandas as pd
from faker import Faker
import random
import openai
import re  # Para extraer números de la respuesta
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# Configurar la API de OpenAI

openai.api_key = "your-api-key"
client = openai.Client()  # Nuevo cliente de OpenAI

In [ ]:
import gradio as gr
import pandas as pd
from faker import Faker
import random
import openai
import re  # Para extraer números de la respuesta
import os  # Para verificar si el archivo se guarda

# Configurar la API de OpenAI (Reemplaza con tu clave real)
openai.api_key = "your-api-key"
client = openai.Client()  # Nuevo cliente de OpenAI

# Crear la instancia de Faker para datos aleatorios
fake = Faker()

# Función para generar datos sintéticos de clientes con simulación de compras y guardarlos en CSV
def generate_customer_data(num_clients=10, save_csv=True):
    data = []
    for _ in range(num_clients):
        name = fake.name()
        email = fake.email()
        age = random.randint(18, 80)
        country = fake.country()
        income = round(random.uniform(20000, 120000), 2)

        # Llamar a la API de OpenAI para generar el gasto mensual estimado
        prompt = f"Dado que un cliente tiene un ingreso anual de ${income}, ¿cuál sería su gasto mensual estimado? Responde con un solo número en dólares."
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente financiero que estima gastos mensuales. Solo responde con el número sin texto adicional."},
                {"role": "user", "content": prompt}
            ]
        )
        response_text = response.choices[0].message.content.strip()

        # Extraer solo el número de la respuesta usando regex
        match = re.search(r"\d+\.?\d*", response_text)
        monthly_spending = float(match.group()) if match else 0.0  # Convertir a float si es posible

        data.append({
            "Name": name,
            "Email": email,
            "Age": age,
            "Country": country,
            "Annual Income ($)": income,
            "Estimated Monthly Spending ($)": monthly_spending
        })

    # Convertir la lista en DataFrame
    df = pd.DataFrame(data)

    # Guardar en un archivo CSV
    if save_csv:
        csv_path = "customer_data.csv"
        df.to_csv(csv_path, index=False)
        print(f"✅ Archivo guardado correctamente en {os.path.abspath(csv_path)}")

    return df


In [20]:
df = generate_customer_data(num_clients=50)  # Puedes cambiar el número de clientes
df.head()  # Muestra las primeras filas del DataFrame


,Name,Email,Age,Country,Annual Income ($),Estimated Monthly Spending ($)
0,Henry Mendoza,davidwright@example.net,34,Paraguay,67865.77,5655.48
1,Jeremy Salas,sheila55@example.org,75,Solomon Islands,67789.04,5632.42
2,Kayla Leach,garymeyer@example.net,63,Burundi,70744.05,5895.34
3,Veronica Rivera,thompsonbrian@example.org,33,San Marino,117012.64,9751.05
4,Stephanie Villanueva,michaelclark@example.net,78,Ecuador,97136.91,8086.41


In [22]:
# Cargar el archivo CSV
df = pd.read_csv("customer_data.csv")

# Mostrar las primeras filas del archivo CSV
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'customer_data.csv'

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df["Annual Income ($)"], bins=20, kde=True)
plt.title("Distribución del Ingreso Anual")
plt.xlabel("Ingreso Anual ($)")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(x=df["Annual Income ($)"], y=df["Estimated Monthly Spending ($)"])
plt.title("Ingreso Anual vs. Gasto Mensual Estimado")
plt.xlabel("Ingreso Anual ($)")
plt.ylabel("Gasto Mensual Estimado ($)")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
df["Age Group"] = pd.cut(df["Age"], bins=[18, 30, 45, 60, 80], labels=["18-30", "31-45", "46-60", "61-80"])
sns.boxplot(x="Age Group", y="Estimated Monthly Spending ($)", data=df)
plt.title("Distribución del Gasto Mensual por Grupo de Edad")
plt.xlabel("Grupo de Edad")
plt.ylabel("Gasto Mensual Estimado ($)")
plt.show()
